In [98]:
import requests
import pandas as pd
import re
import time

In [2]:
tsca = pd.read_excel('tsca_struct_input_080722_v1.xlsx')

In [3]:
tsca

,Unnamed: 0,dtxsid,PREFERRED_NAME,CASRN,INCHIKEY,IUPAC_NAME,SMILES,INCHI_STRING,MOLECULAR_FORMULA,AVERAGE_MASS,...,errors,qsar_ready_smiles,HLC,WS_pred_mol/L,WS_pred_mg/L,test_track,physical_form,NCC,VENDOR_COUNT,IN_SIGMA-ALDRICH
0,0,DTXSID3060164,Benzhydrylbenzene,519-73-3,AAAQKTZKLRYKHR-UHFFFAOYSA-N,"1,1',1''-Methanetriyltribenzene",C1=CC=C(C=C1)C(C1=CC=CC=C1)C1=CC=CC=C1,InChI=1S/C19H16/c1-4-10-16(11-5-1)19(17-12-6-2...,C19H16,244.337,...,NaN,C1=CC=CC=C1C(C1C=CC=CC=1)C1C=CC=CC=1,2.630268e-06,4.073803e-07,0.099452,B,solid,"('Neutral Organics',)",75.0,1.0
1,1,DTXSID7060837,"Propane, 1,3-diiodo-",627-31-6,AAAXMNYUNVCMCJ-UHFFFAOYSA-N,"1,3-Diiodopropane",ICCCI,InChI=1S/C3H6I2/c4-2-1-3-5/h1-3H2,C3H6I2,295.890,...,NaN,ICCCI,2.454709e-03,7.413102e-04,219.321006,B,liquid,NaN,72.0,1.0
2,2,DTXSID9025879,"3,3'-(p-Phenylene)diacrylic acid",16323-43-6,AAFXQFIGKBLKMC-UHFFFAOYSA-N,"3,3'-(1,4-Phenylene)di(prop-2-enoic acid)",OC(=O)C=CC1=CC=C(C=CC(O)=O)C=C1,InChI=1S/C12H10O4/c13-11(14)7-5-9-1-2-10(4-3-9...,C12H10O4,218.208,...,NaN,OC(=O)C=CC1C=CC(C=CC(O)=O)=CC=1,6.025596e-10,9.120108e-03,1988.711764,B,solid,NaN,13.0,0.0
3,3,DTXSID2026282,C.I. Vat brown 3,131-92-0,AAKMSGQPNUGLAZ-UHFFFAOYSA-N,"N,N'-(5,10,15,17-Tetraoxo-10,15,16,17-tetrahyd...",O=C(NC1=CC=CC2=C1C(=O)C1=C(C3=C(C=C1)C1=C(N3)C...,InChI=1S/C42H23N3O6/c46-37-24-14-7-8-15-25(24)...,C42H23N3O6,665.661,...,NaN,O=C(NC1=CC2=C(NC3C4=C(C=CC2=3)C(=O)C2C(=CC=CC=...,2.630268e-09,2.454709e-08,0.016377,B,solid,NaN,13.5,0.0
4,4,DTXSID4052188,"2,5,5-Trimethyl-2-phenyl-1,3-dioxane",5406-58-6,AALXTPRRKXUUOM-UHFFFAOYSA-N,"2,5,5-Trimethyl-2-phenyl-1,3-dioxane",CC1(C)COC(C)(OC1)C1=CC=CC=C1,"InChI=1S/C13H18O2/c1-12(2)9-14-13(3,15-10-12)1...",C13H18O2,206.285,...,NaN,CC1(C)COC(C)(OC1)C1C=CC=CC=1,9.772372e-05,1.148154e-03,236.669687,B,solid,"('Neutral Organics',)",11.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14242,14242,DTXSID5064209,2-(Carboxymethylthio)benzothiazole,6295-57-4,ZZUQWNYNSKJLPI-UHFFFAOYSA-N,"[(1,3-Benzothiazol-2-yl)sulfanyl]acetic acid",OC(=O)CSC1=NC2=CC=CC=C2S1,InChI=1S/C9H7NO2S2/c11-8(12)5-13-9-10-6-3-1-2-...,C9H7NO2S2,225.280,...,NaN,OC(=O)CSC1=NC2=CC=CC=C2S1,3.388442e-09,9.772372e-04,219.870381,B,solid,NaN,79.0,1.0
14243,14243,DTXSID7062873,"Butanoic acid, 4-chloro-, methyl ester",3153-37-5,ZZUYIRISBMWFMV-UHFFFAOYSA-N,Methyl 4-chlorobutanoate,COC(=O)CCCCl,"InChI=1S/C5H9ClO2/c1-8-5(7)3-2-4-6/h2-4H2,1H3",C5H9ClO2,136.580,...,NaN,COC(=O)CCCCl,2.818383e-04,2.041738e-01,27773.578977,B,liquid,"('Esters (Acute toxicity)',)",71.0,1.0
14244,14244,DTXSID9040342,"4-Ethyl-1,3-dioxolan-2-one",4437-85-8,ZZXUZKXVROWEIF-UHFFFAOYSA-N,"4-Ethyl-1,3-dioxolan-2-one",CCC1COC(=O)O1,"InChI=1/C5H8O3/c1-2-4-3-7-5(6)8-4/h4H,2-3H2,1H3",C5H8O3,116.116,...,NaN,CCC1COC(=O)O1,2.570396e-08,9.772372e-01,113405.784063,B,liquid,"('Neutral Organics',)",49.0,1.0
14245,14245,DTXSID3044889,"2,5-Di-tert-butylcyclohexa-2,5-diene-1,4-dione",2460-77-7,ZZYASVWWDLJXIM-UHFFFAOYSA-N,"2,5-Di-tert-butylcyclohexa-2,5-diene-1,4-dione",CC(C)(C)C1=CC(=O)C(=CC1=O)C(C)(C)C,"InChI=1S/C14H20O2/c1-13(2,3)9-7-12(16)10(8-11(...",C14H20O2,220.312,...,NaN,CC(C)(C)C1=CC(=O)C(=CC1=O)C(C)(C)C,5.370318e-04,3.981072e-03,876.418325,B,solid,NaN,61.0,1.0


In [77]:
df_new = tsca[['INCHIKEY']].copy()


In [63]:
df_new = df_new.tail()

In [8]:
df_new

,INCHIKEY
0,AAAQKTZKLRYKHR-UHFFFAOYSA-N
1,AAAXMNYUNVCMCJ-UHFFFAOYSA-N
2,AAFXQFIGKBLKMC-UHFFFAOYSA-N
3,AAKMSGQPNUGLAZ-UHFFFAOYSA-N
4,AALXTPRRKXUUOM-UHFFFAOYSA-N


In [114]:
groups = ['kingdom','superclass','class','subclass']

classy_data = []
for idx, inchikey in df_new['INCHIKEY'].items():
    classyfire = [inchikey]

    response = requests.get(f'http://classyfire.wishartlab.com/entities/{inchikey}.json').json()
    if 'code' in response:
        time.sleep(1)
        response = requests.get(f'http://classyfire.wishartlab.com/entities/{inchikey}.json').json()
        try:
            for group in groups:
                if response[group] is not None:        
                    category_name = response[group]['name']
                    classyfire.append(category_name)
                else:
                    classyfire.append('N/a')
        except:
            classyfire.append('err','err','err','err')
    classy_data.append(classyfire)


In [115]:
df3 = pd.DataFrame(classy_data,columns=['INCHIKEY','Kingdom','Superclass','Class','Subclass'])

In [119]:
df3.head()

,INCHIKEY,Kingdom,Superclass,Class,Subclass
0,AAAQKTZKLRYKHR-UHFFFAOYSA-N,Organic compounds,Benzenoids,Triphenyl compounds,N/a
1,AAAXMNYUNVCMCJ-UHFFFAOYSA-N,Organic compounds,Organohalogen compounds,Organoiodides,N/a
2,AAFXQFIGKBLKMC-UHFFFAOYSA-N,Organic compounds,Phenylpropanoids and polyketides,Cinnamic acids and derivatives,Cinnamic acids
3,AAKMSGQPNUGLAZ-UHFFFAOYSA-N,Organic compounds,Benzenoids,Anthracenes,N/a
4,AALXTPRRKXUUOM-UHFFFAOYSA-N,Organic compounds,Organic oxygen compounds,Organooxygen compounds,Ethers


In [117]:
df3.to_pickle('tsca_classyfire1.pkl')

In [122]:
df3.to_csv('tsca_classyfied.csv')

In [145]:
df3['Kingdom'].astype('str').value_counts()

Organic compounds               13460
Inorganic compounds               593
['err', 'err', 'err', 'err']      194
Name: Kingdom, dtype: int64

In [147]:
df_err = df3[df3['Kingdom'].astype('str').str.contains('err')]

In [148]:
df_err.to_pickle('tsca_err.pkl')